<a href="https://colab.research.google.com/github/rahiakela/edureka-deep-learning-with-tensorflow/blob/module-5-convolutional-neural-networks/module_5_assignment_convolutional_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 5: Convolutional Neural Networks

**Implement convolutional neural-network (CNN) for MNIST Dataset**

Network architecture - CNN with 4 layers has following architecture.

* Input layer : 784 nodes (MNIST images size)
* First convolution layer : 5x5x32
* First max-pooling layer
* Second convolution layer : 5x5x64
* Second max-pooling layer
* Third fully-connected layer : 1024 nodes
* Output layer : 10 nodes (number of class for MNIST)